Evraz AI Challenge
---

29-31 октября 2021 г

**Трек 1: Продуйте металл через Data Science**. 

Разработайте модель, прогнозирующую содержание углерода и температуру чугуна во время процесса продувки металла

https://hackathon.evraz.com/

**Исходные данные**

В рамках этой задачи вам будут даны данные о продувки чугуна в цехе:

- **produv** – Таблица содержит основные параметры продувки - мгновенный расход кислорода и положение (наклон) фурмы
- **lom** – Вместе с чугуном в фурму засыпают лом - это часть технологического процесса. Таблица содержит вес и тип ломов, использованных в каждой плавке
- **plavki** – Основная информация по плавке - характеристики плавки (марка металла, направление разливки) и оборудования
- **sip** – Сыпучие добавки, используемые в технологическом процессе
- **chronom** – хронометраж - время начала и конца различных операций во время плавки
- **chugun** – Химический состав и характеристики чугуна
- **gas** – Информация об анализе отходящих газов
- **target** – целевые значения


- [описание задачи](https://russianhackers.notion.site/1-Data-Science-4cc89ba42de1429bbac316f59bf07a3b)
- [атрибутный состав данных](https://russianhackers.notion.site/a685453e4fde41a098d9ad704d906e21?v=c482eaeb8f3143d58763b4b9008f1fec)

### Light Auto ML

Light Auto ML House prices regression: https://www.kaggle.com/alexryzhkov/lightautoml-houseprices-love

Light Auto ML Titanic classification: https://towardsdatascience.com/lightautoml-preset-usage-tutorial-2cce7da6f936 и https://www.kaggle.com/alexryzhkov/lightautoml-extreme-short-titanic-solution

In [1]:
import pandas as pd
import numpy as np
from pathlib2 import Path
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional
import re
import lightgbm as lgb
from datetime import datetime

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

## Функции

# Загрузка данных

In [2]:
path = Path('../../../data/2021_evraz')

In [3]:
target_train = pd.read_pickle(path.joinpath('target_train_wo_gas_wo_sip.pkl'))
print(target_train.shape)
target_train.head(3)

(2063, 145)


,NPLV,TST,C,VES,T,SI,MN,S,P,CR,...,plavka_TIPE_FUR_koniceskaa,plavka_TIPE_FUR_zilindriceskaa,plavka_TIPE_GOL_4-soplh54,plavka_TIPE_GOL_4-soplx54,plavka_TIPE_GOL_5soplovaa,plavka_TIPE_GOL_601-5,plavka_TIPE_GOL_E32,plavka_TIPE_GOL_E37_4-soplh54,plavka_TIPE_GOL_E_4-soplh54,pol_ras_sum
0,510008,1690,0.060,263700.0,1396.0,0.44,0.22,0.023,0.097,0.03,...,0,1,0,0,1,0,0,0,0,593045.405789
1,510009,1683,0.097,264500.0,1419.0,0.68,0.20,0.017,0.087,0.02,...,0,1,0,0,1,0,0,0,0,567680.040137
2,510010,1662,0.091,263800.0,1384.0,0.56,0.26,0.017,0.096,0.03,...,0,1,0,0,1,0,0,0,0,591476.261946


In [4]:
test = pd.read_pickle(path.joinpath('test_wo_gas_wo_sip.pkl'))
print(test.shape)
test.head(3)

(780, 143)


,NPLV,VES,T,SI,MN,S,P,CR,NI,CU,...,plavka_TIPE_FUR_koniceskaa,plavka_TIPE_FUR_zilindriceskaa,plavka_TIPE_GOL_4-soplh54,plavka_TIPE_GOL_4-soplx54,plavka_TIPE_GOL_5soplovaa,plavka_TIPE_GOL_601-5,plavka_TIPE_GOL_E32,plavka_TIPE_GOL_E37_4-soplh54,plavka_TIPE_GOL_E_4-soplh54,pol_ras_sum
0,512324,240100.0,1355.0,0.46,0.33,0.027,0.079,0.01,0.01,0.02,...,0,1,1,0,0,0,0,0,0,463015.154170
1,512327,266400.0,1390.0,0.30,0.33,0.032,0.099,0.01,0.00,0.00,...,0,1,1,0,0,0,0,0,0,499984.992640
2,512328,270200.0,1373.0,0.00,0.00,0.000,0.000,0.00,0.00,0.00,...,0,1,1,0,0,0,0,0,0,468358.082766


In [5]:
target_train.columns

Index(['NPLV', 'TST', 'C', 'VES', 'T', 'SI', 'MN', 'S', 'P', 'CR',
       ...
       'plavka_TIPE_FUR_koniceskaa', 'plavka_TIPE_FUR_zilindriceskaa',
       'plavka_TIPE_GOL_4-soplh54', 'plavka_TIPE_GOL_4-soplx54',
       'plavka_TIPE_GOL_5soplovaa', 'plavka_TIPE_GOL_601-5',
       'plavka_TIPE_GOL_E32', 'plavka_TIPE_GOL_E37_4-soplh54',
       'plavka_TIPE_GOL_E_4-soplh54', 'pol_ras_sum'],
      dtype='object', length=145)

In [6]:
automl_t = TabularAutoML(task = Task('reg', metric = 'mse'))
automl_c = TabularAutoML(task = Task('reg', metric = 'mse'))

In [7]:
oof_pred_t = automl_t.fit_predict(target_train,  roles = {'target': 'TST', 'drop': 'C'})
oof_pred_c = automl_c.fit_predict(target_train,  roles = {'target': 'C', 'drop': 'TST'})

In [8]:
test_pred_t = automl_t.predict(test)
test_pred_c = automl_c.predict(test)

In [79]:
pd.DataFrame({'NPLV': test['NPLV'],
              'TST':test_pred_t.data[:, 0], 
              'C': test_pred_c.data[:, 0]
             })

,NPLV,TST,C
0,512324,1651.203491,0.033007
1,512327,1656.208740,0.044633
2,512328,1650.830566,0.058527
3,512331,1647.513916,0.034300
4,512333,1653.981567,0.041524
...,...,...,...
775,513369,1665.734009,0.073311
776,513370,1665.300537,0.091745
777,513371,1664.160645,0.069668
778,513372,1670.006836,0.083593


In [80]:
# получаем текущие дату и время
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# создаём путь и имя файла с датой и временем
# file_name = f'../../data/kaggle/gb_competitive_data_analysis/lgb_predictions_{now}.csv'
file_name = f'lama_{now}.csv'
print('File name: ', file_name)

# сохраняем в csv
pd.DataFrame({'NPLV': test['NPLV'],
              'TST':test_pred_t.data[:, 0], 
              'C': test_pred_c.data[:, 0]
             }).to_csv(file_name, index=False, encoding='utf-8')
print('\n File saved to disk!')

File name:  lama_2021-10-31_00-18-27.csv

 File saved to disk!


In [11]:
def metric(answers, user_csv):

    delta_c = np.abs(np.array(answers['C']) - np.array(user_csv['C']))
    hit_rate_c = np.int64(delta_c < 0.02)

    delta_t = np.abs(np.array(answers['TST']) - np.array(user_csv['TST']))
    hit_rate_t = np.int64(delta_t < 20)

    N = np.size(answers['C'])

    return np.sum(hit_rate_c + hit_rate_t) / 2 / N